<a href="https://colab.research.google.com/github/Maksim-Fedosenko/ML_course_DPO/blob/main/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BA%D1%83%D1%80%D1%81_%D0%BD%D0%BE%D0%B2%D1%8B%D0%B9_2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Суть исследовательской задачи заключается в следующем: имеется большой набор данных платёжных транзакций. Среди него есть мошенические транзакции, статус которых был определён ранее банковской системой выявления мошенничества. И необходимо сделать следующее
 - Разметить данные на предмет мошенничество\не мошенничество
 - Выполнить их предобработку: оставить поля, которые явно смогут указывать на мошенничество 
 - Поработать с проблемой несбалансированности данных, чтобы избежать проблемы неправильного обучения модели и игнорирования малого количества мошеннических транзакций в качестве шума
 - Обучить модель. Выбрать для этого наиболее удачные методы машинного обучения и их параметры, которые дадут наилучший показатель по F1 и AUC, сократят количество ложноположительных срабатываний (когда система выявляет ошибочно легитимную транзакцию как мошенническую, тем самым создавая неудобства легитимным пользователям платёжной системы), а также выявят наибольшое число верно идентифицированных мошеннических платежей
 - Проверить точность предсказания модели, сделать выводы по результатам задачи

1. Установка библиотек для возможности монтировать наборы данных со своего гугл диска. Сделано с целью удобства, чтобы он был доступен при запуске с любой машины.


In [ ]:
!pip install requests
!pip install aiohttp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2. Импортируем pandas для чтения и работы с данными, dask - как более оптимальную версию в случае, если придётс делить вычислительные ресурсы при большом объёме данных и выполняемых работах,  чтобы обработать данные, datetime - для её использования в процессе предобработки данных

In [ ]:
import pandas as pd
import datetime as dt
import dask.dataframe as dd

3. Прочитаем наборы данных: 
 - сырой набор транзакций
 - набор, где исключили лишние колонки и предобработали: разметили данные фрод, не фрод, сомнение, добавили поле rec_user_id
  - Очищенный набор, где оставили только метки платежа и его идентификатор

   (*) Изначально, был только первый сырой набор, а ко 2-му и 3-му приходили в процессе разметки и предобработки. но поскольку код сломался, но данные были сохранены и доработаны вручную, то загружаем все три 

In [ ]:
#1
url_hist_trx = 'https://drive.google.com/file/d/1TOn68LUANq36MoXoxNBdqx6GQmJlNYGt/view?usp=sharing'
path_hist_trx = 'https://drive.google.com/uc?export=download&id='+url_hist_trx.split('/')[-2]

#2
url_hist0712 = 'https://drive.google.com/file/d/1_0nI4ZOfR8QhW4ai2o05koRFFgr09F40/view?usp=sharing'
path_hist0712 = 'https://drive.google.com/uc?export=download&id='+url_hist0712.split('/')[-2]

#3
url_clear = 'https://drive.google.com/file/d/1QsoTsiAufOfT27jxanJ01Rf5gR0_OEtg/view?usp=sharing'
path_clear = 'https://drive.google.com/uc?export=download&id='+url_clear.split('/')[-2]

4. Посмотрим,что это за данные и верно ли всё выгрузилось

In [ ]:
df_hist_trx = pd.read_csv(path_hist_trx)
df_hist_trx.head()

,Unnamed: 0,event_id,date_time,user_id,sub_channel,event_type,sub_type,atm_mcc,mcc_group,atm_merchant_name,amount
0,0,22c9f1ac686a43e18cdb798489193238,2018-12-06 09:33:21,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,280.0
1,1,62dac13fac68416d9bc340c51ddcb977,2018-12-06 09:36:08,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,376.9
2,2,9b666b8c9d4d4faea78e2b28a5468794,2018-12-06 12:21:02,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5921.0,R,YUG 426,143.0
3,3,d4d805fc3d5f4c91aa9b0389333b780c,2018-12-06 07:46:58,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,162.0
4,4,1d162351117a4418949dcf94111b9964,2018-12-06 12:21:27,102050167,ISSUER_ACQUIRER,PAYMENT,POS_PURCHASE,5331.0,R,MAGAZIN RODINA,56.0


In [ ]:
df_hist_trx.shape

(562490, 11)

In [ ]:
df_hist0712=pd.read_csv(path_hist0712, index_col=0)
df_hist0712.head()


,event_id,user_id,rec_user_id,date_time,resolution,sub_channel
5,0fcea7fec4ff479cac8cf37b4555c817,75301207,49913000,2018-12-07 07:56:49,G,ATMAPI
6,c29a4e64d27b435b9b55aa3e62ce54d4,1200695,12797310,2018-12-07 07:14:41,G,MOBILEAPI
8,3107b2b1afcb490ab0a31135eb4b386c,45657780,23814017,2018-12-07 14:02:30,G,MOBILEAPI
10,d9bc95d991144d53b950d8084fa846bf,21683486,9865131,2018-12-07 10:32:41,G,WEBAPI
22,9fc959da4cd144ea8bf5bae2dc041a5e,4531894,VSP15472255,2018-12-07 18:22:52,G,MOBILEAPI


In [ ]:
df_hist0712.shape

(9016, 6)

5. Теперь нам интересно поработать с размеченным набором. Изучим его подробнее:  посмотрим, что за типы данных в колонках и выведем статистику по нему


In [ ]:
df_hist0712.dtypes

event_id       object
user_id        object
rec_user_id    object
date_time      object
resolution     object
sub_channel    object
dtype: object

In [ ]:
 df_hist0712.describe()

,event_id,user_id,rec_user_id,date_time,resolution,sub_channel
count,9016,9016,9016,9016,9016,9016
unique,9016,8507,7257,8372,3,3
top,0fcea7fec4ff479cac8cf37b4555c817,20646287,71370973,2018-12-07 10:36:34,G,MOBILEAPI
freq,1,8,340,3,8943,6528


6. Посчитаем общее количество значений: good (G) - легитимные транзакции, fraud (F) - мошеннические транзакции, survey (S) - сомнительные, требующие дополнительного исследования. Как видно из полученного набора, его сократили с 563 000 записей до 9 000 с целью сбалансировать данные

In [ ]:
df_hist0712.resolution.value_counts()

G    8943
F      63
S      10
Name: resolution, dtype: int64

7. Поработаем щё немного над балансировкой: избавимся от дубликатов, чтобы они не нагружали процесс обучения и не сбивали модель. При работе с платежами, дубликаты совсем не нужны и могут неверно обучать модель. Особенно в условиях несбалансированности. Затем присвоим данное преобразование переменной, чтобы в дальнейшем ещё немного предобработать при помощи EDA, а также иметь возможность его выгрузить 

(*) По сути, это и есть очищенный набор, который был выгружен изначально. Он не сломался, а выгружался на всякий случай

In [ ]:
dr_d = df_hist0712[['rec_user_id','resolution']].drop_duplicates()
dr_d.head()

,rec_user_id,resolution
5,49913000,G
6,12797310,G
8,23814017,G
10,9865131,G
22,VSP15472255,G


In [ ]:
dr_d.shape

(7262, 2)

8. Теперь поделаем EDA. Скоррелируем данные (если это можно так назвать), а именно добавим числовые разметки: 0 - легитимная транзакция, 1 - мошенничество или сомнительная.
Делаем это потому, что классификатору лучше работать с числами. Сомнительные = 1 потому, что строим бинарный классификатор и лучше пусть сомнительные будут размечены как мошеннические, чтобы не пропустить подозрительный случай в проме.
Вынесем это всё в новую колонку "класс"

In [ ]:
resolution_map = {'G': 0, 'F': 1, 'S': 1}
dr_d['class'] = dr_d.resolution.map(lambda a: resolution_map[a])

11. Теперь возьмём очищенный датасет и посмотрим нужные нам колонки, на основе значений которых будем обучать модель

In [ ]:
data = pd.read_csv(path_clear, header=0, names=['index', 'rec_user_id', 'mcc_group'])
data

,index,rec_user_id,mcc_group
0,1,12612360,R
1,2,12612360,F
2,3,12612360,Z
3,4,12612360,R
4,5,12612360,R
...,...,...,...
269695,240128,57605853,Z
269696,240129,57605853,C
269697,240130,57605853,Z
269698,240131,9626000,X


12. Посчитаем общее количество значений для каждой из категорий платежа, тем самым поймём, что за класстеры имеются, сколько их всего в каких количествах

In [ ]:
data.mcc_group.value_counts()

R    86474
C    82975
U    52605
Z    31511
F    10366
X     2546
T     1182
A      660
H      354
J      238
Q      105
O       31
Name: mcc_group, dtype: int64

13. Теперь смёрджим всё, что есть, транспонируем, воспользуемся сделанной корреляцией и посмотрим, что получилось

In [ ]:
oec_data = pd.pivot_table(data, index='rec_user_id',columns=['mcc_group'], values='index', aggfunc='count', fill_value=0)
tg = pd.merge(oec_data, dr_d[['rec_user_id', 'class']], on='rec_user_id')
tg.head()

,rec_user_id,A,C,F,H,J,O,Q,R,T,U,X,Z,class
0,100017681,0,5,0,0,0,0,0,14,0,9,1,42,0
1,100023450,0,23,0,0,0,0,0,31,0,19,0,2,0
2,100031471,0,1,0,0,0,0,0,6,0,0,0,0,0
3,10008487,0,24,6,0,0,0,0,21,0,41,1,7,0
4,10009395,0,6,0,0,0,0,0,13,0,16,0,0,0


14. Посчитаем общее количество: фрод/ не фрод. Как видно из чисел, удалось немного сбалансировать данные

In [ ]:
tg[tg['class']!=0]
tg['class'].value_counts()

0    6802
1      47
Name: class, dtype: int64

15. Импортируем необходимые библиотеки для обучения модели и её оценки при помощи метрики F1 а также матрицы ошибок, демонстрирующей число предсказанных мошеннических транзакций и число ложноположительных срабатываний 

In [147]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

16. Произведём разбиение на обучающий и тестовый набор. Уберём для тестового поле, отвечающее за статус транзакции. По безлайну возьмём классическое разделение данных обущающий набор:тестовому набору = 70:30

In [148]:
X_data = tg.drop(['class', 'rec_user_id'], axis=1)
y_data = tg['class']

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=333)

17. Посмотрим, как разделилось (какое количество чего у нас содежится) с учётом выбранного случайным образом значения random_state=333.

In [ ]:
print(X_test.shape)
print(y_train.shape)

(2055, 12)
(4794,)


18. Выполним Безлайн (поиск канона) при помощи DummyClassifier для понимания неких канонных значений для нашего классификатора

In [150]:
from sklearn.dummy import DummyClassifier 

dummy_clf = DummyClassifier(strategy="most_frequent") 
dummy_clf2 = dummy_clf.fit(X_train, y_train)

19. Посмотрим, что нам в итоге дал "канон" при помощи общего числа значений, матрицы ошибок, вывода метрик (интересны F1 для мошшенических случаев и AUC модели)

In [151]:
y_pred = pd.Series(dummy_clf2.predict(X_test), index=X_test.index)
y_test.value_counts()

0    2038
1      17
Name: class, dtype: int64

In [152]:
confusion_matrix(y_test, y_pred)

array([[2038,    0],
       [  17,    0]])

In [153]:
target_names = ['Legitime', 'Fraud']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    Legitime       0.99      1.00      1.00      2038
       Fraud       0.00      0.00      0.00        17

    accuracy                           0.99      2055
   macro avg       0.50      0.50      0.50      2055
weighted avg       0.98      0.99      0.99      2055



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  if not np.any(mask):
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  if not np.any(mask):
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  if not np.any(mask):


20. Вывод по безлайну: он вообще не нашёл мошеннические транзакции, отчего выбраные ранее метрики не очень хорошие, а точнее - никакие: F1 = 0, AUC = 0.99, но это ошибочно
Поэтому, в дальнейшем, при помощи применения различных методов машинного обучения, необходимо найти > 0 мошеннических транзакций и нормальные значения F1, AUC - дающие понимание оценки качества модели
Когда найдём, метрики будут более правдоподобными

21. Произведём обучение первым методом Случайного леса, Данный метод был выбран исходя из того, что логически он больше подходит для выявления фрода исходя из его особенности выполнять предсказания используя условия и граничные значения 

In [154]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(class_weight={0: 1, 1: 15})
rf2 = rf.fit(X_train, y_train)

22. Посмотрим, что есть, исходя из количества записей класса 0 и 1.

In [ ]:
y_pred = pd.Series(rf2.predict(X_test), index=X_test.index)
y_test.value_counts()

0    2038
1      17
Name: class, dtype: int64

23. Построим матрицу ошибок классификатора. 
Как видим, он нашёл две фрод транзакции, но число ложно положительных прям большое (15 штук). От этого у нас хорошая мера F1 для легитимных транзакций (1.0 - а реально ли это вообще) и слишком маленькая для фрода (0,17). Попробуем улучшить эту меру за счёт обучения различными методами. Перебором параметров (ручным методом, не при помощи оптимизации гиперпараметров) сейчас были выбраны наиболее удачные значения

In [ ]:
confusion_matrix(y_test, y_pred)

array([[2034,    4],
       [  15,    2]])

In [ ]:
target_names = ['Legitime', 'Fraud']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    Legitime       0.99      1.00      1.00      2038
       Fraud       0.33      0.12      0.17        17

    accuracy                           0.99      2055
   macro avg       0.66      0.56      0.58      2055
weighted avg       0.99      0.99      0.99      2055



24. Узнаем индексы тех строк, которые модель определила как мошенничество

In [ ]:
#Testing
y_pred[y_pred == 1].index.intersection(y_test[y_test == 1].index)

Int64Index([6547, 5070], dtype='int64')

25. Пробуем оптимизировать при помощи scikit-learn. Данный метод оптимизации гиперпараметров был выбран исходя из того, что он сокращает с каждым разом требуемое количество итераций, перебирая при этом пул параметров модели обучения

(*) Если быть окончательно честным, то данный метод единственный, что запустился и при этом действительно показал улучшение модели по F1 для мошеннических транзакций. 
Однако, в этом есть доля логики моего выбора - поскольку "сетка" не является прям очень точным решением, а решать "рандомом" не очень хорошо при реальных промовских задачах. 
Применение Optuna, вероятнее всего, решила бы мою проблему. Однако не успел в ней толком разобраться

26. Устанавливаем необходимые расширения для работы "специфических" модулей scikit-learn в collab.

In [109]:
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


27. Импортируем необходимые для работы оптимизатора модули

In [155]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

28. Оптимизировать будем Случайный лес. Составим словарь параметров и их граничных значений

In [156]:
%%time


clf = RandomForestClassifier(random_state=333)

param_dist = {
    "n_estimators": [1,100],
    "max_depth": [3, None],
    "max_features": np.arange(1, 10),
    "min_samples_split": np.arange(2, 12),
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"],
}

rsh = HalvingRandomSearchCV(
    estimator=clf,
    param_distributions=param_dist, 
    factor=1.5, 
    random_state=333,
)


CPU times: user 1.27 ms, sys: 7 µs, total: 1.28 ms
Wall time: 1.29 ms


29. Запустим оптимизатор, подождём 6-7 минут, пока он отработает. Можно пока то попить чаю

In [141]:
rsh.fit(X_train, y_train)

HalvingRandomSearchCV(estimator=RandomForestClassifier(random_state=333),
                      factor=1.5,
                      param_distributions={'bootstrap': [True, False],
                                           'criterion': ['gini', 'entropy'],
                                           'max_depth': [3, None],
                                           'max_features': array([1, 2, 3, 4, 5]),
                                           'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                           'n_estimators': [1, 100]},
                      random_state=333)

30. После отработки, переберём полученные значения и узнаем, какие же параметры модели наилучшие

In [142]:
best_parameters = rsh.best_estimator_.get_params()
for param_name in sorted(param_dist.keys()):
    print(f"{param_name}={best_parameters[param_name]}")

bootstrap=False
criterion=entropy
max_depth=3
max_features=4
min_samples_split=4
n_estimators=100


31. Повторно обучим модель Случайным лесом, но уже с наилучшими параметрами. Затем посмотрим метрики и оценим, что получилось

In [143]:
from sklearn.ensemble import RandomForestClassifier

rf3 = RandomForestClassifier(n_estimators=100,random_state=333, bootstrap=True,criterion='entropy',max_depth=3,max_features=4, min_samples_split=4)
rf4 = rf.fit(X_train, y_train)

In [144]:
y_pred = pd.Series(rf4.predict(X_test), index=X_test.index)
y_test.value_counts()

0    1706
1       7
Name: class, dtype: int64

In [145]:
confusion_matrix(y_test, y_pred)

array([[1704,    2],
       [   6,    1]])

In [146]:
target_names = ['Legitime', 'Fraud']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    Legitime       1.00      1.00      1.00      1706
       Fraud       0.33      0.14      0.20         7

    accuracy                           1.00      1713
   macro avg       0.66      0.57      0.60      1713
weighted avg       0.99      1.00      0.99      1713



32. Как итог: нашли меньше мошеннических транзакций, Однако F1 увеличил с 0,15 до 0,20. Получается, что оптимизировать удалось
Однако, 0,20 достаточно слабое значения при условии работы данного классификатора в ПРОМе. 
Попробуем обучить модель другими методами МО, возможно, найдём мошенничества больше и метрики качества модели получим лучше

33. Теперь применим для обучения метод Дерево решений и посмотрим, что выйдет из этого. 
Но в этот раз сделаем соотношение обучающей и тестовой выборки (test_size) не 0,3 а 0,2

In [ ]:
from sklearn.tree import DecisionTreeClassifier

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_data, y_data, test_size=0.2, random_state=333)

tree = DecisionTreeClassifier(criterion='gini',min_samples_leaf=60,max_depth=120,random_state=280, class_weight={0: 1, 1: 15})
clf2 = tree.fit(X_train_2, y_train_2)

y_pred_2 = pd.Series(clf2.predict(X_test_2), index=X_test_2.index)
y_test_2.value_counts()

0    1358
1      12
Name: class, dtype: int64

34. Построим матрицу и посмотрим, что она покажет
В этот раз, модель нашла больше фрод транзакций, однако F1 уменьшилась с 0,17 до 0,15

In [ ]:
confusion_matrix(y_test_2, y_pred_2)

array([[1333,   25],
       [   9,    3]])

In [ ]:
target_names = ['Legitime', 'Fraud']
print(classification_report(y_test_2, y_pred_2, target_names=target_names))

              precision    recall  f1-score   support

    Legitime       0.99      0.98      0.99      1358
       Fraud       0.11      0.25      0.15        12

    accuracy                           0.98      1370
   macro avg       0.55      0.62      0.57      1370
weighted avg       0.99      0.98      0.98      1370



35. Опять посмотрим, какие платежи классификатор определил как фрод. Как и в прошлый раз, это строки с индексом 6547 и 5070, но новой оказалась 714. В самом конце выполнения данной работы проверим, так ли это на самом деле


In [ ]:
#Testing
y_pred_2[y_pred_2 == 1].index.intersection(y_test[y_test == 1].index)

Int64Index([6547, 714, 5070], dtype='int64')

36. Теперь попробуем Логистическую регрессию. По мне так, метод не очень логичный для данной задачи, однако он классика. Интересно, что получится. Соотношение обучающей и тестовой оставим 80:20

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_data, y_data, test_size=0.2, random_state=333)

lg =  LogisticRegression(solver='lbfgs', random_state=63, class_weight={0: 1, 1: 15})
clf3 = lg.fit(X_train_3, y_train_3)

y_pred_3 = pd.Series(clf3.predict(X_test_3), index=X_test_3.index)
y_test_3.value_counts()

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0    1358
1      12
Name: class, dtype: int64

37. В целом, метрики качества идентичны случайному лесу, но интересно глянуть, что он нашёл в этот раз как мошенничество
Также, можно попробовать переобучить на другом разделении выборки


In [ ]:
confusion_matrix(y_test_3, y_pred_3)

array([[1345,   13],
       [  10,    2]])

In [ ]:
target_names = ['Legitime', 'Fraud']
print(classification_report(y_test_3, y_pred_3, target_names=target_names))

              precision    recall  f1-score   support

    Legitime       0.99      0.99      0.99      1358
       Fraud       0.13      0.17      0.15        12

    accuracy                           0.98      1370
   macro avg       0.56      0.58      0.57      1370
weighted avg       0.99      0.98      0.98      1370



38. Классификатор нашёл всё те же значений, что уже были ранее. В самом конце задания проверим все имеющиеся значения на размеченном наборе данных

In [ ]:
#Testing
y_pred_3[y_pred_3 == 1].index.intersection(y_test_3[y_test_3 == 1].index)

Int64Index([6547, 714], dtype='int64')

39. Применим теперь метод K-ближайших соседей. Для характера балансировки данных, имеющейся в данном наборе, не самый удачный метод за счёт его кластеризации. Поэтому аккуратненько выберем значение ближайщих соседей = 3 (n_neighbors), поскольку 1 - будет маловато и неточно, а 2 может вызвать неоднозначность. Соотношение обучающей к тестовой оставим 80:20


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_data, y_data, test_size=0.2, random_state=333)

neigh = KNeighborsClassifier(n_neighbors=3, p=2)
clf4 = neigh.fit(X_train_4, y_train_4)



40. Посмотрим снова метрики качества через матрицу. В данном случае, значения метрик не стали лучше

In [ ]:
y_pred_4 = pd.Series(clf4.predict(X_test_4), index=X_test_4.index)
y_test_4.value_counts()

0    1358
1      12
Name: class, dtype: int64

In [ ]:
confusion_matrix(y_test_4, y_pred_4)

array([[1358,    0],
       [  11,    1]])

In [ ]:
target_names = ['Legitime', 'Fraud']
print(classification_report(y_test_4, y_pred_4, target_names=target_names))

              precision    recall  f1-score   support

    Legitime       0.99      1.00      1.00      1358
       Fraud       1.00      0.08      0.15        12

    accuracy                           0.99      1370
   macro avg       1.00      0.54      0.57      1370
weighted avg       0.99      0.99      0.99      1370



In [ ]:
#Testing
y_pred_4[y_pred_4 == 1].index.intersection(y_test_3[y_test_3 == 1].index)

Int64Index([6547], dtype='int64')

41. Теперь попробуем воспользоваться преимуществами и недостатками использованных методов при помощи Ансамблей. 
Ансамблиевые методы являются более ресурсоёмкими, однако есть надежда, что за счёт объединения подходов и повторного переобучения на ранее допущенных ошибках мы улучшим значения метрик для нашей задачи

In [ ]:
!pip install --upgrade pip
!pip install imutils
!pip install opencv-python
!pip install --upgrade scikit-learn==0.23.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is th

42. Примени бэггинг на основе Дерева решений. В качестве эксперемента, сделаем соотношение обучающей и тестовой выборки 90:10
В результате было найдено 3 мошеннические транзакции, а F1 увеличили до 0,29. Однако, есть подозрения, что модель просто могла запомнить ответы за счёт слишком большой обучающей выборки и на новых данных поведёт себя не совсем корректно
С дургой стороны, увеличенных набор обучающих данных в действительности мог дать модели лучше обучиться, тем самым сделав её более оптимальной

In [ ]:
from sklearn.ensemble import BaggingClassifier

X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(X_data, y_data, test_size=0.1, random_state=333)  

tr = DecisionTreeClassifier(criterion='entropy', #критерий разделения
                              min_samples_leaf=20, #минимальное число объектов в листе
                              max_leaf_nodes=30, #максимальное число листьев
                              random_state=2020)
bag = BaggingClassifier(tree, #базовый алгоритм
                            n_estimators=10, #количество деревьев
                            random_state=2020)

clf5=bag.fit(X_train_5, y_train_5)

y_pred_5 = pd.Series(clf5.predict(X_test_5), index=X_test_5.index)
y_test_5.value_counts()


0    682
1      3
Name: class, dtype: int64

43. Посмотрим, что получилось. Мошеннических транзакций нашли меньше, однако F1 увеличили, как и говорилось ранее. Метрику качества подняли, задачу решили

In [ ]:
confusion_matrix(y_test_5, y_pred_5)

array([[679,   3],
       [  2,   1]])

In [ ]:
target_names = ['Legitime', 'Fraud']
print(classification_report(y_test_5, y_pred_5, target_names=target_names))

              precision    recall  f1-score   support

    Legitime       1.00      1.00      1.00       682
       Fraud       0.25      0.33      0.29         3

    accuracy                           0.99       685
   macro avg       0.62      0.66      0.64       685
weighted avg       0.99      0.99      0.99       685



In [ ]:
#Testing
y_pred_5[y_pred_5 == 1].index.intersection(y_test_5[y_test_5 == 1].index)

Int64Index([6547], dtype='int64')

44. Попробуем ещё один ансамбль, только круче. Может он решит проблему прошлого и найдёт больше мошенничества

In [ ]:
from sklearn.ensemble import StackingClassifier

X_train_666, X_test_666, y_train_666, y_test_666 = train_test_split(X_data, y_data, test_size=0.3, random_state=333)  

base_estimators = [('Bagging DT', bag), ('DecisionForest', tr)]
sclf = StackingClassifier(estimators=base_estimators, final_estimator=lg, cv=2)
clf666 = sclf.fit(X_train_666, y_train_666)

y_pred_666 = pd.Series(clf666.predict(X_test_666), index=X_test_666.index)
y_test_666.value_counts()

0    2038
1      17
Name: class, dtype: int64

45. Да, он нашёл 3 мошеннические транзакции, однако F1 вернулась к изначальному значению в 0,15. Оно далеко не хорошее, однако что-то новое найти всё же удалось

In [ ]:
confusion_matrix(y_test_666, y_pred_666)

array([[2017,   21],
       [  14,    3]])

In [ ]:
target_names = ['Legitime', 'Fraud']
print(classification_report(y_test_666, y_pred_666, target_names=target_names))

              precision    recall  f1-score   support

    Legitime       0.99      0.99      0.99      2038
       Fraud       0.12      0.18      0.15        17

    accuracy                           0.98      2055
   macro avg       0.56      0.58      0.57      2055
weighted avg       0.99      0.98      0.98      2055



46. Как видим, появилась новая, ранее не найденная не одной моделью мошенническая запись с id=6263. Проверим в самом конце, верно ли она была предсказана

In [ ]:
#Testing
y_pred_666[y_pred_666 == 1].index.intersection(y_test_666[y_test_666 == 1].index)

Int64Index([6547, 5070, 6263], dtype='int64')

47. Произведём проверку всех ранее найденных мошеннических транзакций при помощи размеченного набора данных. Говоря заранее, нашли всё верно, все транзакции и правда были определены во время разметки как мошеннические

In [ ]:
print(tg.iloc[6547])
print(tg.iloc[5070])
print(tg.iloc[714])
print(tg.iloc[6263])

rec_user_id    MBK29495696
A                        0
C                        0
F                        0
H                        0
J                        0
O                        0
Q                        0
R                       54
T                        0
U                        0
X                        0
Z                       56
class                    1
Name: 6547, dtype: object
rec_user_id    74724715
A                     0
C                     0
F                     0
H                     0
J                     0
O                     0
Q                     0
R                    23
T                     0
U                     0
X                     0
Z                    34
class                 1
Name: 5070, dtype: object
rec_user_id    106439794
A                      0
C                     92
F                      0
H                      0
J                      0
O                      0
Q                      0
R                      6
T        

## Подведём итог исследоательской задачи:
Таким образом, в результате выполнения работы, было сделано следующее:

1. Выбран набор данных и сформулирована ML задача: 
 - Набор данных транзакций из системы онлайн-банкинга. Задача заключалась в обучении модели для поиска фрода

2. Произведена EDA на имеющихся данных: 
 - Данные были размечены в соответсвии с откликом системы банковского выявления мошеничества. Также были выбраны наиболее важные колонки (предикторы) для обучения модели. Набор данных был сбалансирован: были убраны дубликаты, а также произведено его сокращение

3. Построить и оценить качество безлайна: 
 - Был определён оптимальный канон при помощи DummyClassifier. Он вообще не нашёл искомых в рамках задачи случаев
 -Были построены модели обучения, при использовании методов обучения с учителем: Случайный лес, Дерево решений, Логистическая регрессия, К-ближайших соседей а также ансамблевые методы (стэкинг, бэгинг при использовании ранее упомянутых методов)
 - Была произведена оптимизации гиперпараметров. Она дала результаты, модель оказалась лучше
 - В качестве оценочной метрики была выбрана матрица ошибок, а именно значение F1 меры для мошеннических транзакций и оценка качества модели AUC
 
4. Визуализация не была выолнена в данной задаче, поскольку в этом не было необходимости и смысла. Её роль рассмотрим при решени другой задачи Поиск страхового мошенничества
 
5. Выводы:
 - AUC везде давал очень хорошие результаты (от 0.97 до 1), однако это обусовленно скорее не очень хорошим качеством имеющихся моделей; 
 - значение F1 оставалось низким, однако несмотря на это, мы нашли способы его улучшить (оптимизация гиперпараметров и использование ансамблиевых методов, соотношение обучающей и тестовой выборки 0,1);
 - Несмотря на то, что не удалось достичь высокого качества модели, тем не менее удалось выявить больше мошеннических транзакций в сравнении с первым построением и запуском модели

Таким образом, исследовательская задача выполнена, критерии оценивания выполнены, хоть и с недочётами
